Import der benötigten Module. Das Modul "ecgdetectors" wird zuerst heruntergeladen und installiert.

In [ ]:
from google.colab import files
import csv
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import sklearn
!pip install py-ecg-detectors
from ecgdetectors import Detectors
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

nb_classes = 2

Upload-Dialog, um die EKG-Daten in die virtuelle Maschine zu laden und zu entpacken (es erfolgt keine Prüfung, ob die richtige Datei hochgeladen wurde)

In [ ]:
uploaded = files.upload()
!unzip training.zip

Alternative: direkter Download aus Google Drive (deutlich schneller)

In [3]:
from google.colab import drive
drive.mount('/content/drive')
!cp "/content/drive/My Drive/Wettbewerb_KI/training.zip" .
!unzip training.zip

Mounted at /content/drive
cp: cannot stat '/content/drive/My Drive/Wettbewerb_KI/training.zip': No such file or directory
unzip:  cannot find or open training.zip, training.zip.zip or training.zip.ZIP.


In [ ]:
sdnn_normal = np.array([])                                # Initialisierung der Feature-Arrays
sdnn_afib = np.array([])
ecg_class = np.array([])
with open('training/REFERENCE.csv') as csv_file:      # Einlesen der Liste mit Dateinamen und Zuordnung
    csv_reader = csv.reader(csv_file, delimiter=',')
    normal_line_count = 0
    afib_line_count = 0
    for row in csv_reader:
      data = sio.loadmat('training/'+row[0]+'.mat')   # Import der EKG-Dateien
      ecg_lead = data['val'][0]
      if ecg_lead.size>=9000:                                   #discard data smaller than 9000 and trim data to same size
        if row[1] != 'A':                                       # includes N,O,~
          if normal_line_count != 0:
            sdnn_normal = np.vstack((sdnn_normal,ecg_lead[0:9000]))        
          else:
            sdnn_normal=ecg_lead[0:9000]
          normal_line_count = normal_line_count + 1    
        if row[1]=='A':                                         # Zuordnung zu "Vorhofflimmern"          
          if afib_line_count != 0:
            sdnn_afib = np.vstack((sdnn_afib,ecg_lead[0:9000]))        
          else:
            sdnn_afib=ecg_lead[0:9000]                                
          afib_line_count = afib_line_count + 1

    if (normal_line_count+ afib_line_count % 100)==0:
        print(str(normal_line_count+ afib_line_count) + "\t Dateien wurden verarbeitet.")


In [ ]:
X=np.concatenate((sdnn_normal, sdnn_afib)) #combine Normal and Afib Data

normal_class=np.zeros(sdnn_normal.shape[0])       #combine class labels
normal_class.fill(0)
afib_class=np.zeros(sdnn_afib.shape[0])
afib_class.fill(1)
y=np.concatenate((normal_class, afib_class))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)#split data for testing and training

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)